# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import pandas as pd
import numpy as np
import re

### obtain the data that is in the table of postal codes and transform the data into a pandas dataframe

In [2]:
wikidata = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", 
             match='Postcode', header=0)[0]

### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [3]:
wikidata = wikidata.rename(
    mapper={'Postcode':'PostalCode', 'Neighbourhood':'Neighborhood'},
    axis=1)
wikidata.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [4]:
befrows = wikidata.shape[0]
wikidata = wikidata[wikidata['Borough']!='Not assigned']
assert wikidata.shape[0] < befrows, "No rows were removed"

### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma 

In [5]:
grouped = wikidata.groupby(['PostalCode','Borough'], 
                           as_index=False).aggregate(lambda x: ', '.join(x))

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [6]:
for index, row in grouped.iterrows():
    row['Neighborhood'] = row['Borough'] if re.search('Not assigned', row['Neighborhood']) else row['Neighborhood']

### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [7]:
grouped.shape

(103, 3)

# Section 2: Geocoder

In [8]:
import geocoder

In [9]:
geodata = pd.read_csv("https://cocl.us/Geospatial_data").rename(
    mapper={"Postal Code": "PostalCode"}, axis=1)

In [10]:
geodata['PostalCode'] = geodata['PostalCode'].apply(str)
grouped['PostalCode'] = grouped['PostalCode'].apply(str)
merged = grouped.merge(geodata, on=['PostalCode'])
merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Section 3: Explore and cluster neighborhoods

### You can decide to work with only boroughs that contain the word Toronto

In [11]:
tordata = merged[merged['Borough'].str.contains('Toronto')]

### Then replicate the same analysis we did to the New York City data.

In [12]:
import folium
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [13]:
address = 'Toronto, ONT'
geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#adjust
longitude +=.25
print(f'The geograpical coordinate of Toronto are {latitude}, {longitude}.')
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, borough, neighborhood in zip(tordata['Latitude'], 
                                           tordata['Longitude'], 
                                           tordata['Borough'], 
                                           tordata['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

The geograpical coordinate of Toronto are 43.6781255, -79.3821235335026.


### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Credentials

In [14]:
with open('/Users/kutch/Desktop/IBM/foursquarecreds.txt') as f:
    fsq = f.read()
client_id = re.search("(?<=CLIENT_ID)[ =]+'([A-Z0-9]+)", fsq).group(1)
client_secret = re.search("(?<=CLIENT_SECRET)[ =]+'([A-Z0-9]+)", fsq).group(1)
#not sure if coursera wants me to update this or not
VERSION = '20180605' # Foursquare API version

In [15]:
#from foursquare
#function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Get all venues

In [16]:
#for a given list of neighborhoods
def get_nearby_venues(neighborhoods, latitudes, longitudes):
    radius, limit = 500, 100
    global client_id, client_secret 
    venues_list=[]
    for name, lat, lng in zip(neighborhoods, latitudes, longitudes):
        # create the API request URL
        url = f'https://api.foursquare.com/v2/venues/explore?&client_id={client_id}&client_secret={client_secret}&v={VERSION}&ll={lat},{lng}&radius={radius}&limit={limit}'
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

toronto_venues = get_nearby_venues(neighborhoods=tordata['Neighborhood'], latitudes=tordata['Latitude'], longitudes=tordata['Longitude'])

#### Categorize venues

In [17]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot = toronto_onehot.drop(columns=['Neighborhood'], axis=1)
toronto_onehot.insert(loc=0, 
                      column='Neighborhood', 
                      value=toronto_venues['Neighborhood'])
#find mean frequency of categories by neighborhood
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

#### Most common categories per neighborhood

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]
def get_common_venues(location_df):
    num_top_venues = 10
    indicators = {1:'st', 2:'nd', 3:'rd'}
    indicators.update(dict((el,'th') for el in range(4, num_top_venues+1)))
    cols = ['Neighborhood']
    for num, ind in indicators.items():
        cols.append(f'{num}{ind} Most Common Venue')
    neighborhoods_venues_sorted = pd.DataFrame(columns=cols)
    neighborhoods_venues_sorted['Neighborhood'] = location_df['Neighborhood']
    for ind in np.arange(location_df.shape[0]):
        neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(location_df.iloc[ind, :], num_top_venues)
    return neighborhoods_venues_sorted

neighborhoods_venues_sorted = get_common_venues(toronto_grouped)

#### Cluster model

In [19]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = tordata
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'),on='Neighborhood')

#### Visualize the clusters

In [20]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Observation: it seems as if the downtown area is dominated by Cluster 3. Consider refining the sample to a more specific area to parse out heterogenity within this group.